# Tekstikolletsioonid ja nende kasutamine

Tekstikollekstsioonide (*korpuste*)  töötlemiseks on olemas kaks põhimõttelist võimalust:

* tekstid on salvestatud failidena (*pickle*),
* tekstid on salvestatud andmebaasi objektidena.

Andmebaasi kasutamise eeliseks on parem otsitavus ning võimalus kasutada olemasolevaid algoritme.

* EstNLTK 1.4 teek kasutab [ElasticSearch](https://www.elastic.co/products/elasticsearch) otsingumootorit teksikollektsioonide salvestamiseks. 
* EstNLTK 1.6 teek kasutab [PostgreSQL](https://www.postgresql.org) andmebaasi teksikollektsioonide salvestamiseks. 

PostgreSQL eeliseks:

* stabiilne API,
* ennustatav resursi kulu,
* parem skaleeruvus praktiliste andmemahtude juures,
* lihtsam integreeritavus olemasolevasse it-taristusse.

### Tüüpilised sammud tekstikollektsioonide töötamisel 
* Tekstikollektsiooni loomine
* Tekstikollektsiooni annoteerimine
* Tekstikollektsioonidest otsimine


## 0. Ettevalmistavad sammud 

Selleks et Jupyteriga oleks lihtsam töötada tuleks soovitusi muuta agresiivsemaks (*autocomplete*).

In [1]:
%config IPCompleter.greedy=True

See võimaldab meil *Tab*-iga küsida objekti meetodeid ning *Shift+Tab*-iga küsida funktsioonide dokumentatsiooni.

## 1. Tekstikollektsioonide loomine failidest

Selleks et oleks selge, mis mooduleid konkreetsetes sammudes kasutatakse, impordime moodulid vajaduse põhiselt iga sammu ees.

### Sisendtektide uurimine

Järgnevas on meie eesmärk uurida raadiosaadete automaat-transkriptsiooni abil saadud materjale. Vastavad failid asuvad kataloogis `data/kpt` ning on UTF-8 kodeeringus. Nende lugemiseks on vaja fail õiges kodeeringus avada.
Üldiselt pole UTF-8 avamiseks vaja erilisi liigutusi teha, aga üldiselt kasutatakse selleks `codec` teeki.  

In [2]:
import codecs
f1 = codecs.open("data/kpt/2019-09-07.txt", "r", "utf-8")
raw_text = f1.read()
print(raw_text[:300])

K01: Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?
-:
K02: Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Pal


Teksti vaadates on selge, et iga repliik algab kõneleja nimega, millele järgneb repliik. 
Selline struktuur on väha sagedane ka ametlike dokumentide või tootekirjelduste korral. 
Ikka kasutatakse dokumendi eri osade eraldamiseks pealkirju või muid korduvaid struktuurielemente. 
Tavaliselt on kõige lihtsam selliseid struktuurielemente tuvastada regulaaravaldiste abil.  

## Teksti esmane segmenteerimine

Esimese asjana tuleks tekstist leida üles kõnetuvastaja poolt pandud rääkija tähis või nimi. Selleks kasutame kahte märgendajat:
* ```ŖegexTagger``` abil märgime peale rääkija
* ```TextSegmentsTagger``` abil märgime peale rääkija kõneldud laused 

### Kõnelejale vastav regulaaravaldis

Kuna kõnetuvastuse väljund on selgete struktuuriga, siis on vastava regulaaravaldise leidmine üsna lihtne.
Sama olukord on ka keerukamate poolstruktureeritud dokumentide segmenteerimisel. Tüüpiliselt on üsna lihtne leida korduvaid mustreid, mis defineerivad erinevate tekstide osasid (lõikude nummerdus, kuupäevaline kirje päis, jms).

Igal juhul on vastava regulaaravaldise tuletamiseks ja testimiseks vaja võtta illustreerivad näited. Nendest saab hiljem luua ka testi, mille abil tulevikus tagada funktsionaalsuse säilimine mustrite täiendamisel.

In [3]:
import re

In [4]:
test_text =  'K01: Kuku...\n-:\nArtur Talvik: Tere, siin on...'

Kuna regulaaravaldistest arusaamine on keerukas, siis teeme seda sammhaaval. 
Selleks, et mitte takerduda paosümbolite (*escape symbols*) rägastikku, on mõistlik kasutada `r'sõnesid'`.
Nii ei ole vaja regulaaravalidiste paosümboleid mitmekordselt pagendada.

In [5]:
print(re.findall(r'^.',test_text))
print(re.findall(r'((^|\n).)',test_text))
print(re.findall(r'((^|\n).*:)',test_text))

['K']
[('K', ''), ('\n-', '\n'), ('\nA', '\n')]
[('K01:', ''), ('\n-:', '\n'), ('\nArtur Talvik:', '\n')]


Et edaspidi oleks lihtsam, tuleks regulaaravaldise osadele anda nimed:
* ```name``` - kõneleja nimi,
* ```span``` - kõneleja fikseeriv tekstifragment.

In [6]:
pattern = r'((^|\n)(?P<span>(?P<name>.*): ?))'
print([match.group('name') for match in re.finditer(pattern, test_text)])
print([match.group('span') for match in re.finditer(pattern, test_text)])
print([match.group() for match in re.finditer(pattern, test_text)])

['K01', '-', 'Artur Talvik']
['K01: ', '-:', 'Artur Talvik: ']
['K01: ', '\n-:', '\nArtur Talvik: ']


### RegexTagger mustrisõnastik

Märgendaja ```RegexTagger``` defineerimiseks on peale mustrite vaja veel spetsifitseerida mitu olulist asja:
* ```_regex_pattern_``` - regulaaravaldis,
* ```_group_``` - *spani* defineeriv regulaaravaldise grupp,
* ```_priority_``` - mustri prioriteet konfliktide korral,
* ```_validator_``` - validaator valepositiivsete vastete eemldamiseks.

Neist esimesed kaks on alati vajalikud ning ülejäänud on kasulikud enamikel juhtudel. Meie näites on meil märgendamiseks vajalik vaid üks muster. 

In [7]:
header_voc_1 = \
[{
    '_regex_pattern_': pattern,
    '_group_': 'span',
    '_priority_': 1,
}]

Lisaks on mõistlik defineerida ka *spanidele* vastavad annotatsiooni atribuudid. Meie näites siis kõneleja isik. Selles tuleb  anda ette funktsioon, mis võttab sisse kogu regulaaravalidsele vastava ```Match``` objekti ja tuletab sellest atribuudi väärtuse.  

In [8]:
header_voc_2 = \
[{
    '_regex_pattern_': pattern,
    '_group_': 'span',
    '_priority_': 1,
    'person': lambda m: m.group('name') 
}]

In [9]:
header_voc_3 = \
[{
    '_regex_pattern_': pattern,
    '_group_': 'span',
    '_priority_': 1,
    '_validator_': lambda m: m.group('span') != '-:', 
    'person': lambda m: m.group('name') 
}]

### RegexTagger loomine

Märgendaja `RegexTagger` loomisel on tarvis alati määrata peamised parameetrid:

* `vocabulary` - regulaaravaldiste mustrite sõnastik,
* `output_layer` - väljundkihi nimi,
* `output_attributes` - väljundkihi tellitavate atribuutide nimed.

Lisaks saab mängida erinevate täpishäälestust võimaldavate parameetritega:
* `ambiguous` - kas leitud märgendused võib olla mitu annotatsiooni,
* `conflict_resolving_strategy` - mida teha kui regulaaravalistele vastavad fragmendid on ülekattes 
* `overlapped, ignore_case` - lisaargumendid argumendid `re.finditer` funktsiooni häälestamiseks

Üldiselt on täpishäälestuse parameetreid vaja vaid siis, kui mustrisõnastik käitub ootamatult ning on tarvis aru saada, mis läks valesti.  

In [10]:
from estnltk import Text
from estnltk.taggers import RegexTagger

Märgendaja ei pea annoteerima valitud teksti 

In [11]:
tagger = RegexTagger(vocabulary = header_voc_1, output_layer = 'headers') 
text = tagger.tag(Text(raw_text))
display(text.headers[:5])

Layer(name='headers', attributes=(), spans=SL[Span('K01: ', [{}]),
Span('-:', [{}]),
Span('K02: ', [{}]),
Span('-:', [{}]),
Span('K06: ', [{}])])

Selleks, et et märgendaja annoteeriks teksti peab:

* mustrisõnastikus olema reegel atribuudi arvutamiseks,
* vastav atribuut peab olema tellitud väljundkihti.

In [12]:
# Atribuudid pole tellitud
tagger = RegexTagger(vocabulary = header_voc_2, output_layer = 'headers') 
text = tagger.tag(Text(raw_text))
display(text.headers[:3])

# Atribuut person on tellitud 
tagger = RegexTagger(
    vocabulary = header_voc_2, 
    output_layer = 'headers',
    output_attributes = ['person']
) 

text = tagger.tag(Text(raw_text))
display(text.headers[:3])

Layer(name='headers', attributes=(), spans=SL[Span('K01: ', [{}]),
Span('-:', [{}]),
Span('K02: ', [{}])])

Layer(name='headers', attributes=('person',), spans=SL[Span('K01: ', [{'person': 'K01'}]),
Span('-:', [{'person': '-'}]),
Span('K02: ', [{'person': 'K02'}])])

### Teksti segmenteerimine erinevate inimeste kõneks

Järgmiseks loomulikuks sammuks on repliikide eraldamine üldtekstist. Selle käigus on mõistlik kõneleja isik panna eraldi atribuudiks.
Jällegi on tegemist standardse dokumendi struktureerimise ülesandega, mille käigus jagatakse põhitekst päiste (*header*) järgi osadeks. 

Selle jaoks on EstNLTK teegis olemas `TextSegmentsTagger` märgendaja, mille loomisel on tarvis määrata peamised parameetrid:

* `input_layer` - päiselementide kiht
* `output_layer` - väljundkiht
* `output_attributes` - väljundkihti tellitavate atribuutide nimekiri

In [13]:
from estnltk.taggers import TextSegmentsTagger

In [14]:
tagger = TextSegmentsTagger(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'])

tagger.tag(text)
text.lines[:5]

Layer(name='lines', attributes=('person',), spans=SL[Span('Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n', [{'person': None}]),
Span('\n', [{'person': None}]),
Span('Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Palun lenda kõrge. Võtta asja kerge, tappa maha, ära päädhabi.\n', [{'person': None}]),
Span('\n', [{'person': None}]),
Span('Ta oli liiga uudis ainult koduvabariigist.\n', [{'person': None}])])

Saadud tulemus on vigane:

* kuigi me oleme tellinud väljundkihti kõneleja nime ei jõua selle väärtus atribuutide hulka;
* repliikide hulka on sattunud palju tühje tekste

Nende probleemide lahendamiseks tuleb meil määrata märgendaja `TextSegmentsTagger` parameetrid:

* `decorator` - funktsioon tellitud attribuutide väärtuste arvutamiseks,
* `validator` - funktsioon päiste täiendavaks valideerimiseks.

Viimaks saab parameetriga 

* `include_header` - määrata kas päiselement kuulub tekstiosa koosseisu või mitte. 

Tüüpiliselt on mõistlik päisest mõelda kui tekstiosale vastavast metainfost ja seega see ei peaks olema teksti osa. 

In [15]:
tagger = TextSegmentsTagger(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'],
    decorator = lambda header_span: {'person': header_span['person']},
    validator = lambda header_span: header_span['person'] != '-' 
)
if 'lines' in text.layers:
    del text.lines
tagger.tag(text)
display(text.lines[:5])
display(text.lines[0])

Layer(name='lines', attributes=('person',), spans=SL[Span('Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n-:\n', [{'person': 'K01'}]),
Span('Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Palun lenda kõrge. Võtta asja kerge, tappa maha, ära päädhabi.\n-:\n', [{'person': 'K02'}]),
Span('Ta oli liiga uudis ainult koduvabariigist.\n', [{'person': 'K06'}]),
Span('Tere, siin on Keskpäevatund ja Kuku raadio, Tallinna stuudios Urmas Jaagant, Ainar Ruussaar ja Priit Hõbemägi. Mart Sander laulis lõbusa laulukese. Palun lenda sirgelt ja see laul on meie poolt tervituseks Gunnar kobinile. Kes kelle kohta hiljuti saime teada, et ta loobus Nordika juhi ametikohast, nimelt ta pidi Kobin alustama teisel septembril tordika juhina, aga ootamatult selgus, et ta on juba suvel oma ametikohast loobunud ja tõi ühe põhjusena välja selle, et ühiskonnas tekkinud suur diskussioon tema töötasu üle, mis oli siis viisteist tuhat bruto ühes kuus ei ole vastavuses tema tegeliku töö panusena. Et seda tema lauset oskan mina kahte moodi analüüsida, üks on see, et liiga, et talle makstakse liiga palju raha ja tema panuseks väikeseks, aga teisest küljest võiks mõelda ka niimoodi, et ta ütleb, et selle panuse eest, mis tema suudaks ettevõtte arengusse panna, on seda raha liiga vähe. Kuidas see asi tegelikult on, eks teab ta ise.\n', [{'person': 'Artur Talvik'}]),
Span('Ma arvan, et asi on lihtne nagu tavapäraselt, et Gunnar Kobin ilmselt läks süsteemi mille esiesmapilgul seal tundmaõppimisel said aru, et tema natuur, tema eesmärgid ja see süsteem, kuhu ta sattus ei klapi päris hästi kokku.\n', [{'person': 'Ainar Ruussaar'}])])

text,person
Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n-:\n,K01


Saadud tulemus on ikkagi vigane, kuna päiselementide ignoreerimine suureneab vaid tekstilõikude pikkust ning ei jäta vastatatele päiselementidele vastavaid tekstiosasid välja. Seega tuleb esmalt need tekstiosad siiski sisse jätta. 

In [16]:
tagger = TextSegmentsTagger(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'],
    decorator = lambda header_span: {'person': header_span['person']}
)
if 'lines' in text.layers:
    del text.lines
tagger.tag(text)
display(text.lines[:5])
display(text.lines[0])

Layer(name='lines', attributes=('person',), spans=SL[Span('Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n', [{'person': 'K01'}]),
Span('\n', [{'person': '-'}]),
Span('Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Palun lenda kõrge. Võtta asja kerge, tappa maha, ära päädhabi.\n', [{'person': 'K02'}]),
Span('\n', [{'person': '-'}]),
Span('Ta oli liiga uudis ainult koduvabariigist.\n', [{'person': 'K06'}])])

text,person
Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n,K01


### Segmentatsiooni esmane valideerimine

Esmase korrektsuse hindamiseks uurime millised on kõnelejad. Selleks kasutame kihi grupeerimisfunktsiooni. 
Vastav funktsionaalsus kopeerib mooduli `Pandas` grupeerimisloogikat, mis on de facto `GROUP BY` analoog Pyhtonis. Esimene näide illustreerib gruppide suuruse leidmist.

In [17]:
text.lines.groupby(['person']).count

{('K01',): 1,
 ('-',): 3,
 ('K02',): 1,
 ('K06',): 3,
 ('Artur Talvik',): 21,
 ('Ainar Ruussaar',): 16,
 ('K07',): 13,
 ('Ignar Fjuk',): 2}

Järgnev näide võimaldab uurida millised repliigid vastavad kõnelejale `K06`. 

In [18]:
for span in text.lines.groupby(['person']).groups[('K06',)]:
    print(span.text)


Ta oli liiga uudis ainult koduvabariigist.

Ta on tähtajaline neli päevauudiseid koduvabariigist.

Ta oli neli päevauudiseid koduvabariigist.



Viimane kõige keerukam näide võimaldab leida repliikide kogupikkuse tähemärkides. 

In [19]:
display(text.lines.groupby(['person'])
      .aggregate(func = lambda spans: sum(s.end - s.start for s in spans)))

{('K01',): 104,
 ('-',): 2,
 ('K02',): 243,
 ('K06',): 140,
 ('Artur Talvik',): 19145,
 ('Ainar Ruussaar',): 9171,
 ('K07',): 12607,
 ('Ignar Fjuk',): 30}

Kuna Keskpäevatunnis on reeglina kolm osalejat, siis võib arvata, et märgend `K07` vastab Ignar Fjukile.

## 2. Tekstikollektsiooni loomine

Järgnevas näites lisame kõikidele repliikidele vastavad tekstiobjektid Postgre andmebaasis olevasse tekstikollektsiooni. 

### Ühenduse loomine Postgre SQL andmebaasiga 

EstNLTK teegis defineeritakse ühenduse parameetrid läbi `PostgreStorage` klassi.  

In [20]:
from estnltk.storage.postgres import PostgresStorage

In [21]:
storage = PostgresStorage(host='127.0.0.1',
                          port=5432,
                          dbname='ekt',
                          user='swen',
                          password='kala',
                          schema='media_analysis',
                          role=None,
                          temporary=False)

INFO:storage.py:42: connecting to host: '127.0.0.1', port: 5432, dbname: 'ekt', user: 'swen'
INFO:storage.py:58: schema: 'media_analysis', temporary: False, role: 'swen'


Kui andmebaasis parameetritele vastavat skeemat pole siis tuleb see luua. 
Analoogselt saab ka kogu skeema kustutada.

In [22]:
from estnltk.storage.postgres import create_schema, delete_schema

In [23]:
# create_schema(storage)
# delete_schema(storage)

Lisaks saab vaadata ka skeemasse kuuluvadi kollektsioone ning ühenduse parameetreid.

In [24]:
storage

### Tekstikollektsiooni formaadi fikseerimine

Tekstikollektsiooni saab lisaks tekstidele salvestada ka tekstidele vastavaid metaandmeid. 
Metaandmeid hoitakse eraldiseisvates veergudes ning edaspidi on nende järgi lihtne ja kiire tekste grupeerida.
Meie näite korral on teksti kirjeldavateks olulisteks metaatribuutideks:

* kõneleja isik, 
* saate toimumise aeg.

Neist esimene on sõne ja teine kuupäeva formaadis. Need andmeformaadid on EstNLTK poolt toetatud. 

In [25]:
# Metaatribuutide kirjeldus
storage['kpt'].meta = {'date':'datetime', 'person':'str'}
# Kollektsiooni formaadi fikseerimine
storage['kpt'].create('Keskpäevatunni saadete automaatne transcriptsioon')

INFO:collection.py:107: new empty collection 'kpt' created


,data type
date,timestamp without time zone
person,text


Implementatsioonivea tõttu ei jäta tekstikollektsioon meelde tellitud metaatribuute ning need tuleb eraldi tellida. 

In [26]:
print(storage['kpt'].column_names)
storage['kpt'].column_names += list(storage['kpt'].meta.keys())
print(storage['kpt'].column_names)

['id', 'data']
['id', 'data', 'date', 'person']


### Tekstide lisamine kollektsiooni

Selleks pole tarvis midagi muud kui itereerida üle kõikide andmefailide:

* luua esmane tekst objekt;
* märgendada igale tekstile repliigid 
* luua iga repliigi korral tekstobjekt ning paigutada see kollektsiooni.

Eelnevalt on kogu vajalik töö ära tehtud nüüd on vaja vaid kogu kood kokku koguda.

In [27]:
import os
from datetime import datetime

In [28]:
header_voc_2 = \
[{
    '_regex_pattern_': pattern,
    '_group_': 'span',
    '_priority_': 1,
    'person': lambda m: m.group('name') 
}]

header_tagger = RegexTagger(
    vocabulary = header_voc_2, 
    output_layer = 'headers', 
    output_attributes = ['person'])

In [29]:
segmenter = TextSegmentsTagger(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'],
    decorator = lambda span: {'person': span['person']},
)

In [30]:
collection = storage['kpt']
directory = os.fsencode('data/kpt')

with collection.insert() as collection_insert:

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename == 'readme.md':
            continue
        
        print(filename)
        date = datetime.strptime(re.search(r'(?P<date>.*)\.txt', filename).group('date'),'%Y-%m-%d')
        file = codecs.open("data/kpt/{}".format(filename), "r", "utf-8")
        text = segmenter(header_tagger(Text(file.read())))
    
        for span in text.lines:
            
            line = span.text.strip()
            if line == '':
                continue
                
            line_text = Text(line)
            line_text.tag_layer(['compound_tokens', 'words', 'paragraphs'])
            meta_data = {'date': date, 'person': span['person']}
            collection_insert(text=line_text, meta_data=meta_data)

2019-09-07.txt
2019-10-26.txt
2019-10-19.txt
2019-09-14.txt
2019-09-28.txt
2019-10-12.txt
2019-10-05.txt
2019-09-21.txt
INFO:collection.py:325: inserted 463 texts into the collection 'kpt'


## 3. Tekstikollektsiooni annoteerimine

### Tekstikollektsiooni märgenduskihtide tüübid 

Märgenduskihid tekstikollektsioonis jagunevad kaheks:

* esialgsed kihid (luuakse koos tekstiobjektiga).
* eraldi seisvad kihid (neid saab pärast juurde teha).

Kuna esialgsete kihtide muutmine eeldab tekstiobjekti muutmist, siis on mõistliklisada tekstile vaid need kihid, mida edasiste analüüsidega ei muudeta. Kõik ülejäänid kihid tuleks lisadada hiljem eraldi. Meie näites on esialgseteks kihtideks vaid segmentatsioonikihid, mida on vaja morfoloogiliseks analüüsiks. Kõik ülejäänud kihid lisame tekstidele hiljem juurde.

Esialgseid kihte on võimalk vaadata vaadates kollektsiooni kuuluvat teksti.

In [31]:
collection[0]

Text(text='Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?')

### Uue kihi lisamine tekstikollektsiooni

Tekstikollektsiooni uue kihi lisamikseks on kõige lihtsam viis, vajaliku märgendaja loomine ning selle rakendamine kogu kollektsioonile. 
Meie näiteks on selleks vabamorfile vastav märgendaja.

In [32]:
from estnltk.taggers import VabamorfTagger 

In [33]:
vabamorf = VabamorfTagger(disambiguate = True, output_layer = 'morph_analysis')
collection.create_layer(tagger=vabamorf)

INFO:collection.py:817: collection: 'kpt'
INFO:collection.py:836: preparing to create a new layer: 'morph_analysis'
INFO:collection.py:869: inserting data into the 'morph_analysis' layer table
INFO:collection.py:904: layer created: 'morph_analysis'


Tulemust saab vaadata kollektsiooni vaadates. Sealjuures ei ole ilma täiendavate sammudeta kihti võimalik näha.

In [34]:
display(collection)
display(collection[0])

Text(text='Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?')

### Naiivne nimeolemi tuvastus

Järgnevas näitame kuidas saab repliikide peale märkida mõned olulised nimeolemid, mille sagedust saaks näiteks hiljem ajas jälgida. Meie eesmärgiks on siin vaid süsteemi tutvustamine mitte parim täpsus. 

In [35]:
text = Text(raw_text).analyse('morphology')
text.morph_analysis[:5]

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('K01', [{'normalized_text': 'K01', 'lemma': 'K01', 'root': 'K01', 'root_tokens': ['K01'], 'ending': '0', 'clitic': '', 'form': '?', 'partofspeech': 'Y'}]),
Span(':', [{'normalized_text': ':', 'lemma': ':', 'root': ':', 'root_tokens': [':'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('Kuku', [{'normalized_text': 'Kuku', 'lemma': 'Kuku', 'root': 'Kuku', 'root_tokens': ['Kuku'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'H'}]),
Span('raadios', [{'normalized_text': 'raadios', 'lemma': 'raadio', 'root': 'raadio', 'root_tokens': ['raadio'], 'ending': 's', 'clitic': '', 'form': 'sg in', 'partofspeech': 'S'}]),
Span('välja', [{'normalized_text': 'välja', 'lemma': 'välja', 'root': 'välja', 'root_tokens': ['välja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}])])

Vaatame sagedasi lemmasid tuvastamaks oluliste fraaside põhju.

In [36]:
lemma_freq = text.morph_analysis.groupby(['lemma']).count
sorted(lemma_freq.items(), key=lambda x: x[1], reverse=True)

[((',',), 877),
 (('see',), 439),
 (('olema',), 376),
 (('et',), 335),
 (('ja',), 191),
 (('.',), 158),
 (('siis',), 133),
 (('tema',), 115),
 (('mina',), 111),
 (('mis',), 107),
 (('ei',), 103),
 (('kui',), 94),
 (('nagu',), 81),
 (('aga',), 74),
 (('saama',), 68),
 ((':',), 60),
 (('ka',), 58),
 (('nii',), 54),
 (('või',), 53),
 (('asi',), 52),
 (('üks',), 46),
 (('ütlema',), 45),
 (('selline',), 45),
 (('raha',), 40),
 (('pidama',), 38),
 (('tegema',), 35),
 (('nüüd',), 33),
 (('tulema',), 33),
 (('teine',), 31),
 (('kes',), 30),
 (('noh',), 29),
 (('väga',), 28),
 (('teadma',), 26),
 (('võima',), 24),
 (('inimene',), 24),
 (('kas',), 24),
 (('tuhat',), 23),
 (('kuidas',), 23),
 (('mingisugune',), 22),
 (('Artur',), 21),
 (('Talvik',), 21),
 (('veel',), 21),
 (('Euroopa',), 21),
 (('no',), 21),
 (('kõik',), 21),
 (('välja',), 20),
 (('sina',), 20),
 (('juba',), 20),
 (('suur',), 20),
 (('mingi',), 20),
 (('Ainar',), 19),
 (('Ruussaar',), 19),
 (('seal',), 19),
 (('tegelikult',), 18)

In [37]:
from estnltk.taggers import PhraseTagger

In [38]:
entity_voc=[
{
    '_phrase_': ('kuku', 'raadio'),
    'entity': 'Kuku Raadio'
},
{
    '_phrase_': ('keskerakond',),
    'entity': 'Keskerakond' 
},
{
    '_phrase_': ('Keskerakond',),
    'entity': 'Keskerakond' 
},

{
    '_phrase_': ('Eesti',),
    'entity': 'Eesti'
},
{
    '_phrase_': ('Ukraina',),
    'entity': 'Ukraina'
}   
]

In [39]:
entity_tagger=PhraseTagger(
    output_layer='entities',
    output_attributes=['entity'],
    input_layer='morph_analysis', 
    input_attribute='lemma',
    vocabulary=entity_voc, 
    key='_phrase_')

In [40]:
if 'entities' in text.layers:
    del text.entities
entity_tagger(text)
text.entities.groupby(['entity']).count

{('Kuku Raadio',): 3, ('Eesti',): 17, ('Keskerakond',): 17}

Lets run the corresponding tagger on the entire text collection.

In [41]:
collection.create_layer(tagger=entity_tagger)
collection

INFO:collection.py:817: collection: 'kpt'
INFO:collection.py:836: preparing to create a new layer: 'entities'
INFO:collection.py:869: inserting data into the 'entities' layer table
INFO:collection.py:904: layer created: 'entities'


### Tulemuste kiire vaatamine

Selleks et tulemust näha, tuleb eraldi loodud kiht lülitada tekstobjekti rekonstrueerimisel kasutatavate kihtide loetellu. 

In [42]:
collection.selected_layers.append('entities')
collection.selected_layers

['tokens', 'compound_tokens', 'words', 'sentences', 'paragraphs', 'entities']

In [43]:
display(collection[0].text)
display(collection[0].entities)

'Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?'

Layer(name='entities', attributes=('entity',), spans=SL[EnvelopingSpan(['Kuku', 'raadio'], [{'entity': 'Kuku Raadio'}]),
EnvelopingSpan(['Kuku', 'raadiot'], [{'entity': 'Kuku Raadio'}])])

## 4. Päringud tekstikollektsiooni

Päringuid on vaja selleks, et valida välja teatud kriteeriumidele vastavad tekstid. 
Selleks on vaja luua päringuobjekt ning rakendada seda kogu kollektsioonile või alamkollektsioonile.
Erinevaid päringuobjekte on mitmeid ning neid saab kombineerida. 

### Kihipäringud
Vaatame esmalt kihipäringuid, et tuvastada, millised nimeolemid esinevad Kuku Raadioga samas repliigis.  

In [44]:
from estnltk.storage.postgres import JsonbLayerQuery

In [45]:
layer_query ={'entities': JsonbLayerQuery('entities', entity='Kuku Raadio')}

In [46]:
for key, text, meta in collection.select(
    layer_query=layer_query,
    layers=['entities'],
    return_index=True,
    collection_meta=['person', 'date']):
    display(text.entities)

Layer(name='entities', attributes=('entity',), spans=SL[EnvelopingSpan(['Kuku', 'raadio'], [{'entity': 'Kuku Raadio'}]),
EnvelopingSpan(['Kuku', 'raadiot'], [{'entity': 'Kuku Raadio'}])])

Layer(name='entities', attributes=('entity',), spans=SL[EnvelopingSpan(['Kuku', 'raadio'], [{'entity': 'Kuku Raadio'}])])

Layer(name='entities', attributes=('entity',), spans=SL[EnvelopingSpan(['Kuku', 'raadio'], [{'entity': 'Kuku Raadio'}]),
EnvelopingSpan(['Kuku', 'raadiot'], [{'entity': 'Kuku Raadio'}])])

Layer(name='entities', attributes=('entity',), spans=SL[EnvelopingSpan(['Kuku', 'raadio'], [{'entity': 'Kuku Raadio'}]),
EnvelopingSpan(['Eesti'], [{'entity': 'Eesti'}])])

Layer(name='entities', attributes=('entity',), spans=SL[EnvelopingSpan(['Kuku', 'raadio'], [{'entity': 'Kuku Raadio'}])])

...

Layer(name='entities', attributes=('entity',), spans=SL[EnvelopingSpan(['Kuku', 'raadio'], [{'entity': 'Kuku Raadio'}])])

Layer(name='entities', attributes=('entity',), spans=SL[EnvelopingSpan(['Kuku', 'raadio'], [{'entity': 'Kuku Raadio'}]),
EnvelopingSpan(['Kuku', 'raadiot'], [{'entity': 'Kuku Raadio'}])])

Layer(name='entities', attributes=('entity',), spans=SL[EnvelopingSpan(['Kuku', 'raadio'], [{'entity': 'Kuku Raadio'}]),
EnvelopingSpan(['Eesti'], [{'entity': 'Eesti'}]),
EnvelopingSpan(['Eesti'], [{'entity': 'Eesti'}])])

Layer(name='entities', attributes=('entity',), spans=SL[EnvelopingSpan(['Kuku', 'raadio'], [{'entity': 'Kuku Raadio'}]),
EnvelopingSpan(['Kuku', 'raadiot'], [{'entity': 'Kuku Raadio'}])])

Layer(name='entities', attributes=('entity',), spans=SL[EnvelopingSpan(['Kuku', 'raadio'], [{'entity': 'Kuku Raadio'}]),
EnvelopingSpan(['Ukraina'], [{'entity': 'Ukraina'}]),
EnvelopingSpan(['Eestis'], [{'entity': 'Eesti'}]),
EnvelopingSpan(['Eestisse'], [{'entity': 'Eesti'}]),
EnvelopingSpan(['Ukraina'], [{'entity': 'Ukraina'}]),
EnvelopingSpan(['Ukraina'], [{'entity': 'Ukraina'}]),
EnvelopingSpan(['Ukrainast'], [{'entity': 'Ukraina'}])])

### Päringud originaalsele tekstiobjektile

Esialgsetele kihtidele saab teha päringuid läbi tekstipäringute, aga siis peavad vastavad kihid sisaldama atribuute. Meie ühelgil kihil pole atribuute, kuna `text` pole päris atribuut. Seega ei saa me ühtegi huvitavat päringut teha.

In [47]:
from estnltk.storage.postgres import JsonbTextQuery

In [49]:
collection.select(query=JsonbTextQuery('tokens', text='raadios')).head()

[]

### Päringud alustekstile

Leidub eraldi päringuobjekt alusteksti alamsõnede otsimiseks. Seda on kasulik kasutada otsingufaasis.

In [50]:
from estnltk.storage.postgres import SubstringQuery

In [51]:
collection.select(SubstringQuery('kuku')).head(2)


[(13,
  Text(text='Aga praegu peab vist Nordikat iseloomustada nii nagu aastaid tagasi. Noorem laps oli veel väike ja ma käisin temaga koos kinos vaatamas multikaid siis üks multikas Madagaskar, selle üks stseen, kus Pidviinidest lendurid teatasid reisijatele, et meil on teile hea uudis ja halb uudis, hea uudis on see, et me maandume, halb uudis on see, et me kukume, nagu kyll.')),
 (48,
  Text(text='Keskpäevatund jätkab Urmas Jaagant, Ainar Ruussaar ja Priit Hõbemägi Tallinna stuudios. Brexiti on Ühendkuningriigi lahkumine Euroopa Liidust või keegi veel seda ei tea, aga need, kes seda teavad, need on võinud selle nädala jooksul jälgida maailma parimad TV-d raamat, mis on siis vaadeldav nii BBC koduleheküljelt kui Briti parlamendi koduleheküljelt, kui ka kõikidest suurematest maailma infoportaalidest, mis räägivad ja mis näitab otseülekandes seda, kuidas Briti parlament vaidleb selle üle, kas britid peaksid kukkuma kolmekümne esimesel oktoobril kolksti Euroopa Liidust välja ilma igasug

## 5. Andmebaasiühenduse sulgemine

Ühenduse sulgemine.

In [52]:
storage.close()

Käsud Postgre objektide kustutamiseks.

In [53]:
# storage.delete('kpt')
# storage['kpt'].delete()